<a href="https://colab.research.google.com/github/KonradGonrad/PyTorch-deep-learning/blob/main/06_PyTorch_transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
try:
  import torch
  import torchvision
  assert (int(torch.__version__.split('.')[0]) > 1) or \
          (int(torch.__version__.split('.')[0]) == 1 and int(torch.__version__.split('.')[1]) >= 12), 'torch must have newer version than 1.12'
  assert (int(torchvision.__version__.split('.')[1]) >= 13), 'torchvision must be newer version than 0.13'
except:
  print(f'[INFO] torch/torchvision version not as required. Installing newer version')
  !pip3 install -U torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113
  import torch
  import torchvision
  print(f'torch version: {torch.__version__}')
  print(f'torchvision version: {torchvision.__version__}')

In [ ]:
import os
import zipfile
import matplotlib.pyplot as plt
import requests

from pathlib import Path

DATA_PATH = Path('data/')
IMAGE_PATH = DATA_PATH / 'pizza_steak_sushi'

if IMAGE_PATH.is_dir():
  print(f"{IMAGE_PATH} directory already exists")
else:
  print(f"Did not find {IMAGE_PATH} directory, creating one...")
  IMAGE_PATH.mkdir(parents=True, exist_ok=True)

  with open(DATA_PATH / 'pizza_steak_sushi.zip', 'wb') as f:
    request = requests.get('https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip')
    f.write(request.content)

  with zipfile.ZipFile(DATA_PATH / 'pizza_steak_sushi.zip', 'r') as zipf:
    zipf.extractall(IMAGE_PATH)

  os.remove(DATA_PATH / 'pizza_steak_sushi.zip')


In [ ]:
TRAIN_DIR = IMAGE_PATH / 'train'
TEST_DIR = IMAGE_PATH / 'test'

In [ ]:
Additional_func_path = Path('add/')

if Additional_func_path.is_dir():
  print(f'{Additional_func_path} already exists.')
else:
  print(f"Can't find {Additional_func_path}. Creating one..")
  Additional_func_path.mkdir(parents=True, exist_ok = True)
  with open(Additional_func_path / 'notebook_5.ipynb', 'wb') as f:
    request = requests.get('https://github.com/KonradGonrad/PyTorch-deep-learning/raw/main/Kopia_notatnika_05_pytorch_going_modular_exercise_template.ipynb')

    f.write(request.content)

In [ ]:
ADD_FUNS_PATH = Path('Additional_functions/')

if ADD_FUNS_PATH.is_dir():
  print(f'{ADD_FUNS_PATH} path is already created')
else:
  print(f'Creating {ADD_FUNS_PATH} path')
  ADD_FUNS_PATH.mkdir(parents=True, exist_ok=True)

  urls = ['https://github.com/KonradGonrad/PyTorch-deep-learning/raw/main/additional_functions/data_setup.py',
          'https://github.com/KonradGonrad/PyTorch-deep-learning/raw/main/additional_functions/engine.py',
          'https://github.com/KonradGonrad/PyTorch-deep-learning/raw/main/additional_functions/get_data.py',
          'https://github.com/KonradGonrad/PyTorch-deep-learning/raw/main/additional_functions/model_builder.py',
          'https://github.com/KonradGonrad/PyTorch-deep-learning/raw/main/additional_functions/utils.py']

  for url in urls:
    local_filename = url.split('/')[-1]
    request = requests.get(url)
    with open(ADD_FUNS_PATH / local_filename, 'wb') as f:
      f.write(request.content)

In [ ]:
import os
import sys

sys.path.append(str(ADD_FUNS_PATH))
sys.path

In [ ]:
from model_builder import TinyVGG

torch.manual_seed(42)

model_1 =TinyVGG(input_channels = 3,
                 hidden_channels = 10,
                 output_channels = 3)

dummy = torch.randn(size=(1, 3, 64, 64))
torch.argmax(torch.softmax(model_1(dummy), dim=1), dim=1)

In [ ]:
from torchvision import transforms
torch.manual_seed(42)

manual_transform = transforms.Compose([
    transforms.Resize(size=(64, 64)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

In [ ]:
from data_setup import prepare_data

prepared_data = prepare_data(transforms=manual_transform,
                             train_dir = TRAIN_DIR,
                             test_dir = TEST_DIR,
                             batch_size = 1)

train_dataloader, test_dataloader = prepared_data['train_dataloader'], prepared_data['test_dataloader']
class_names = prepared_data['class_names']

In [ ]:
from torchvision import models

weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT
weights

In [ ]:
auto_transforms = weights.transforms()
auto_transforms

In [ ]:
prepared_data = prepare_data(transforms=auto_transforms,
                             train_dir = TRAIN_DIR,
                             test_dir = TEST_DIR,
                             batch_size=32)

train_dataloader = prepared_data['train_dataloader']
test_dataloader = prepared_data['test_dataloader']
class_names = prepared_data['class_names']

train_dataloader, test_dataloader, class_names

In [ ]:
weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT
model = torchvision.models.efficientnet_b0(weights=weights).to('cpu')

In [ ]:
!pip install torchinfo
from torchinfo import summary

summary(model=model,
        input_size=(32, 3, 224, 224),
        col_names=['input_size', 'output_size', 'num_params', 'trainable'],
        col_width = 20,
        row_settings = ['var_names']
        )

In [ ]:
for param in model.features.parameters():
  param.requires_grad = False

In [ ]:
torch.manual_seed(42)
torch.cuda.manual_seed(42)

output_shape = len(class_names)

model.classifier = torch.nn.Sequential(
    torch.nn.Dropout(p = 0.2, inplace=True),
    torch.nn.Linear(in_features=1280,
                    out_features = output_shape,
                    bias=True)
).to('cpu')

In [ ]:
summary(model=model,
        input_size=(32, 3, 224, 224),
        verbose=0,
        col_names=['input_size', 'output_size', 'num_params', 'trainable'],
        col_width = 20,
        row_settings=['var_names'])

In [ ]:
from torch import nn

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
from engine import train
from timeit import default_timer as timer

start_time = timer()

results = train(model=model,
                test_dataloader = test_dataloader,
                train_dataloader = train_dataloader,
                optimizer = optimizer,
                loss_fn = loss_fn,
                epochs = 6,
                device = 'cpu')

end_time = timer()
print(f"[INFO] total training time: {end_time - start_time:.3f} seconds")

In [ ]:
results

In [ ]:
def plot_loss_curves(results):
  train_loss, test_loss = results['train_loss'], results['test_loss']
  train_acc, test_acc = results['train_acc'], results['test_acc']

  epochs = range(len(results['train_loss']))

  plt.figure(figsize=(15, 7))

  plt.subplot(1, 2, 1)
  plt.plot(epochs, train_loss, label='train_loss')
  plt.plot(epochs, test_loss, label='test_loss')
  plt.title('Loss (lower = better)')
  plt.xlabel('Epochs')
  plt.legend()

  plt.subplot(1, 2, 2)
  plt.plot(epochs, train_acc, label='train_accuracy')
  plt.plot(epochs, test_acc, label='test_accuracy')
  plt.title('Accuracy (higher = better)')
  plt.xlabel('Epochs')
  plt.legend()

In [ ]:
plot_loss_curves(results)

In [ ]:
from typing import List, Tuple

from PIL import Image

def pred_and_plot_image(model: torch.nn.Module,
                        image_path: str,
                        class_names: List[str],
                        device: torch.device,
                        image_size: Tuple[int, int] = (224, 224),
                        transform: torchvision.transforms = None
                        ):
  image = Image.open(fp=image_path)

  if transform is not None:
    image_transform = transform
  else:
    image_transform = transforms.Compose([
        transforms.Resize(image_size),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])

  model.to(device)

  model.eval()
  with torch.inference_mode():
    transformed_image = image_transform(image).unsqueeze(dim=0)
    image_pred = model(transformed_image.to(device))

    image_pred_prob = torch.softmax(image_pred, dim=1)
    image_pred_label = torch.argmax(image_pred_prob, dim=1).item()

  plt.figure()
  plt.imshow(image)
  plt.axis('off')
  if class_names is None:
    title = f'It\'s {image_pred_label}'
  else:
    title = f'It\'s {class_names[image_pred_label]} | Probability: {image_pred_prob.max():.2f}'
  plt.title(title)
  plt.show()

In [ ]:
import random

num_images_to_plot = 3
test_image_path_list = list(Path(TEST_DIR).glob('*/*.jpg'))
random_image = random.sample(population=test_image_path_list,
                             k=num_images_to_plot)

for image_path in random_image:
  pred_and_plot_image(model=model,
                      image_path = image_path,
                      class_names=class_names,
                      device='cpu',
                      transform=weights.transforms(),
                      image_size=(224, 224))

In [ ]:
CUSTOM_IMAGE_PATH = Path('custom_image/')

if CUSTOM_IMAGE_PATH.is_dir():
  print(f'Can\'t find {CUSTOM_IMAGE_PATH} path.')
else:
  CUSTOM_IMAGE_PATH.mkdir()
  with open(CUSTOM_IMAGE_PATH / 'pizza.jpeg', 'wb') as f:
    request = requests.get('https://github.com/KonradGonrad/PyTorch-deep-learning/raw/main/additional/pizza.jpeg')
    f.write(request.content)

In [ ]:
pred_and_plot_image(model=model,
                    image_path=CUSTOM_IMAGE_PATH / 'pizza.jpeg',
                    class_names=class_names,
                    device='cpu')